# Importación de liberías

In [1]:
import fastf1

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sys
sys.path.append('../')

from src.circuit_clustering_model.extract import *

In [28]:
# Config less verbosity
fastf1.set_log_level('ERROR')

# Carga de datos

Aquí tendremos que leer el csv con todos los circuitos y para cada circuito necesitamos cargar la sesión con la vuelta de qualy

In [4]:
df = pd.read_csv('../data/output/races.csv', index_col=0)

In [9]:
df = df.loc[:, ['season', 'round', 'circuitId']]

Eliminaríamos duplicados, pero mejor ver los dataframes por si alguna sesión ha transcurrido en lluvia o con neumático que no sea `SOFT`. Los eliminamos al final

In [29]:
from tqdm import tqdm

In [37]:
import pandas as pd
from tqdm import tqdm

def extract_races_and_results_dataframes(races):
    # Verificar si las columnas requeridas existen
    required_columns = {'season', 'round', 'circuitId'}
    if not required_columns.issubset(races.columns):
        raise ValueError(f"El DataFrame 'races' debe contener las columnas {required_columns}")

    # Seleccionar columnas específicas
    races = races.loc[:, ['season', 'round', 'circuitId']]

    # Diccionario para almacenar la información de circuitos
    circuits = {}

    # Iterar sobre las filas del DataFrame
    for season, rnd, circuit_id in tqdm(races.itertuples(index=False), total=len(races), desc="Procesando carreras"):
        try:
            # Obtener información del circuito
            circuits[circuit_id] = get_circuit_info(season, rnd)
        except Exception as e:
            print(f"Error procesando circuito {circuit_id} (temporada {season}, ronda {rnd}): {e}")
            circuits[circuit_id] = None

    # Crear DataFrame a partir del diccionario
    data = pd.DataFrame.from_dict(circuits, orient='index')

    return data


In [38]:
extract_races_and_results_dataframes(df.head())

Procesando carreras: 100%|██████████| 5/5 [00:09<00:00,  1.93s/it]


,compound,laptime,max_speed,distance,n_corners,avg_corner_speed,avg_speed,throttle_perc,brake_perc,gear_changes
bahrain,SOFT,88.997,311.0,5391.107500,15,174.591753,218.342318,73.134771,18.059299,55.0
imola,SOFT,74.411,290.0,4899.232778,19,214.538960,237.668831,81.048701,13.961039,40.0
portimao,MEDIUM,77.968,314.0,4637.664722,15,188.740533,214.136223,73.058824,17.647059,40.0
catalunya,SOFT,76.741,313.0,4665.113611,16,177.564519,219.068966,75.727273,16.300940,40.0
monaco,SOFT,70.346,279.0,3276.551111,19,146.219753,168.225256,59.266212,26.279863,48.0


In [30]:
def extract_races_and_results_dataframes(races):

    races = races.loc[:, ['season', 'round', 'circuitId']]

    # Empty dictionary
    circuits = {}

    for race in tqdm(races.itertuples()):

        _, season, rnd, circuit_id = race
        
        circuits[circuit_id] = get_circuit_info(season, rnd)

    data = pd.DataFrame(circuits).T

    return data

21it [00:27,  1.33s/it]


¿Qué información necesitamos?

* Tiempo por vuelta `lap.LapTime`

* Velocidad máxima en speedtrap `lap.SpeedST`

* Distancia

* Número de curvas

* Velocidad media de paso por curva

* Velocidad media

* Porcentaje de acelerador

* Porcentaje de freno

* Cambios de marcha

Obtenemos directamente la información de todo lo necesario y con esto ya podríamos construir el modelo

Nota: deberíamos considerar eliminar las curvas que sean a fondo y/o agrupar las curvas o rectas por velocidad

Esto son next steps. Hacer si da tiempo